# InfoGraph Tutorial
#### This tutorial illustrates the use of HeCo algorithm [Self-supervised Heterogeneous Graph Neural Network with Co-contrastive Learning](https://arxiv.org/abs/2105.09111), an self-supervised cross-view contrastive learning method on heterogeneous graphs,  which enables the two views to collaboratively supervise each other and finally learn high-level node embeddings.
#### The tutorial is organized as folows:
#### 1. Preprocessing Data and Loading Configuration
#### 2. Training the model
#### 3. Evaluating the model

## 1. Preprocessing Data and Loading Configuration 
#### First, we load the configuration from yml file and the dataset. 
#### For easy usage, we conduct experiments to search for the best parameter across four datasets and find the proper value of parameters such that the performance of implemented HeCo is similar to the value reported in the original paper.

In [13]:
from torch_geometric.loader import DataLoader
import os
from src.methods import HeCo, Sc_encoder, Mp_encoder, HeCoDBLPTransform
from src.trainer import SimpleTrainer
from src.evaluation import LogisticRegression
from src.datasets import DBLP, aminer, ACM, FreebaseMovies
from src.evaluation import LogisticRegression
from src.utils import create_data
from src.config import load_yaml


config = load_yaml('./configuration/heco_acm.yml')
# config = load_yaml('./configuration/heco_dblp.yml')
# config = load_yaml('./configuration/heco_freebase_movies.yml')
# config = load_yaml('./configuration/heco_aminer.yml')
device = config.device

# -------------------- Data --------------------
current_folder = os.path.abspath('')
path = os.path.join(current_folder, config.dataset.root, config.dataset.name)
if config.dataset.name == 'acm':
    dataset = ACM(root=path)
elif config.dataset.name == 'dblp':
    dataset = DBLP(root=path, pre_transform=HeCoDBLPTransform())
elif config.dataset.name == 'freebase_movies':
    dataset = FreebaseMovies(root=path)
elif config.dataset.name == 'aminer':
    dataset = aminer(root=path)
else:
    raise NotImplementedError

data_loader = DataLoader(dataset)

## 2. Training the Model
#### In the second step, we first initialize the parameters of HeCo. There are two encoders for HeCo, specifically encoder for meta-path view and encoder for network schema view.  
#### You may replace the any of the two encoders with the user-defined encoder. Please refer to the framework of the encoder in [heco.py](https://github.com/IDEA-ISAIL/ssl/edit/heco/src/methods/heco.py). Keep in mind that the encoders consist of class initialization and forward function. If you want to revise HeCo, make sure to include get_embs() function in your implementation.

In [5]:
# ------------------- Method -----------------
encoder1 = Mp_encoder(P=config.dataset.P, hidden_dim=config.model.hidden_dim, attn_drop=config.model.attn_drop)
encoder2 = Sc_encoder(hidden_dim=64, sample_rate=config.model.sample_rate, nei_num=config.dataset.nei_num, attn_drop=config.model.attn_drop)

feats = data_loader.dataset._data['feats']
feats_dim_list = [i.shape[1] for i in feats]
method = HeCo(encoder1=encoder1, encoder2=encoder2, feats_dim_list = feats_dim_list, feat_drop=config.model.feat_drop, tau=config.model.tau)
method.cuda()

HeCo()

#### We train the model by calling the trainer.train() function.

In [6]:
trainer = SimpleTrainer(method=method, data_loader=data_loader, device=device, n_epochs=config.optim.max_epoch, lr=config.optim.lr, patience=config.optim.patience)
trainer.train()

mp  [0.5213805  0.47861952]
sc  [1.]
Epoch 0: loss: 6.6926, time: 2.1983s
mp  [0.5035674  0.49643257]
sc  [1.]
Epoch 1: loss: 6.6919, time: 0.1287s
mp  [0.5149494  0.48505065]
sc  [1.]
Epoch 2: loss: 6.6913, time: 0.1213s
mp  [0.48829818 0.5117018 ]
sc  [1.]
Epoch 3: loss: 6.6902, time: 0.1245s
mp  [0.51507056 0.4849294 ]
sc  [1.]
Epoch 4: loss: 6.6880, time: 0.1277s
mp  [0.50593895 0.49406105]
sc  [1.]
Epoch 5: loss: 6.6852, time: 0.1232s
mp  [0.4995878 0.5004122]
sc  [1.]
Epoch 6: loss: 6.6809, time: 0.1184s
mp  [0.48569477 0.51430523]
sc  [1.]
Epoch 7: loss: 6.6755, time: 0.1281s
mp  [0.4877871 0.5122129]
sc  [1.]
Epoch 8: loss: 6.6682, time: 0.1959s
mp  [0.50029135 0.49970862]
sc  [1.]
Epoch 9: loss: 6.6595, time: 0.1261s
mp  [0.5111601  0.48883992]
sc  [1.]
Epoch 10: loss: 6.6478, time: 0.1242s
mp  [0.49217847 0.5078215 ]
sc  [1.]
Epoch 11: loss: 6.6369, time: 0.1275s
mp  [0.47469464 0.5253054 ]
sc  [1.]
Epoch 12: loss: 6.6237, time: 0.1215s
mp  [0.4846989 0.5153011]
sc  [1.]
Epoc

## 3. Evaluating the performance of InfoGraph
#### In the last step, we evaluate the performance of InfoGraph. We first get the embedding by calling method.get_embs() function and then use logistic regression to evaluate its performance. 
#### The more choice of classifiers can be found in [classifier.py](https://github.com/IDEA-ISAIL/ssl/edit/molecure/src/evaluation/classifier.py), including SVM, RandomForest, etc. 
#### Besides, other evaluation methods in an unsupervised setting could be found in [cluster.py](https://github.com/IDEA-ISAIL/ssl/edit/molecure/src/evaluation/cluster.py) or [sim_search.py](https://github.com/IDEA-ISAIL/ssl/edit/molecure/src/evaluation/sim_search.py), including K-means method or similarity search.

In [12]:
# ------------------ Evaluator -------------------
method.eval()
data_pyg = dataset._data[config.dataset.target_type].to(method.device)
embs = method.get_embs(data_loader.dataset._data['feats'], data_loader.dataset._data['mps']).detach()

lg = LogisticRegression(lr=config.classifier.base_lr, weight_decay=config.classifier.weight_decay,
                        max_iter=config.classifier.max_epoch, n_run=config.classifier.n_run, device=device)

data_pyq = create_data.create_masks(data_pyg.cpu())
lg(embs=embs, dataset=data_pyg)

mp  [0.59217894 0.40782106]
Evaluate node classification results
** Val: 86.0980 (1.8035) | Test: 86.1386 (1.3546) **


In [15]:
# ------------------ Optionally Save the Embedding Result -------------------
import pickle as pkl
f = open(os.path.join(current_folder, config.dataset.name + "_embeddings.pkl"),  "wb")
pkl.dump(embs.cpu().data.numpy(), f)
f.close()